# Experiment 12: Keystone Edge Analysis

**Phase 4 - Identifying Critical Network Connections**

## Background

Experiment 8 showed an unexpected finding:
- **Random fragmentation** creates 17.4% tip/recovery asymmetry
- **Targeted high-betweenness removal** creates only 12.3% asymmetry

This suggests that critical recovery pathways may be **distributed throughout the network** rather than concentrated in high-betweenness hubs.

## Key Question

**Which specific edges are most critical for maintaining recovery capacity?**

We systematically remove individual edges and measure the impact on:
1. Tip/recovery ratio (asymmetry)
2. Recovery fraction
3. Total entropy production

## Experimental Design

| Parameter | Value |
|-----------|-------|
| Network | 50-cell Amazon subnetwork |
| Edges to test | Top 100 by flow (or all if fewer) |
| Runs per edge removal | 10 |
| Control (intact network) | 50 runs |
| **Total simulations** | ~1,050 |

## Expected Outputs

1. Ranked list of "keystone" edges whose removal most impacts recovery
2. Network visualization showing critical connections
3. Edge characteristics that predict criticality (flow, betweenness, etc.)

## 1. Setup and Imports

In [ ]:
import sys
sys.path.insert(0, '/opt/research-local/src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import time
from pathlib import Path
from netCDF4 import Dataset
from dask.distributed import as_completed
from collections import defaultdict

# Core energy-constrained module
from energy_constrained import (
    EnergyConstrainedNetwork,
    EnergyConstrainedCusp,
    GradientDrivenCoupling,
    run_two_phase_experiment,
    get_dask_client,
    compute_network_metrics
)

print("Imports successful!")
print(f"NumPy version: {np.__version__}")

In [ ]:
# Connect to Dask cluster
client = get_dask_client()
print(f"Connected to: {client.scheduler_info()['address']}")
print(f"Workers: {len(client.scheduler_info()['workers'])}")
print(f"Total threads: {sum(w['nthreads'] for w in client.scheduler_info()['workers'].values())}")

## 2. Load Amazon Data

In [ ]:
DATA_PATH = Path('/opt/research-local/data/amazon/amazon_adaptation_model/average_network/era5_new_network_data')

def load_amazon_data(year=2003, months=[7, 8, 9]):
    """Load and average Amazon moisture recycling data."""
    all_rain = []
    all_evap = []
    all_network = []
    
    for month in months:
        file_path = DATA_PATH / f'1deg_{year}_{month:02d}.nc'
        if file_path.exists():
            with Dataset(file_path, 'r') as ds:
                all_rain.append(ds.variables['rain'][:])
                all_evap.append(ds.variables['evap'][:])
                all_network.append(ds.variables['network'][:])
    
    return {
        'rain': np.mean(all_rain, axis=0),
        'evap': np.mean(all_evap, axis=0),
        'network': np.mean(all_network, axis=0),
        'n_cells': len(all_rain[0])
    }

amazon_data = load_amazon_data(year=2003)
print(f"Loaded Amazon data: {amazon_data['n_cells']} cells")

## 3. Experiment Configuration

In [ ]:
# Experiment 12: Keystone Edge Analysis

SWEEP_CONFIG = {
    # Edge selection
    'max_edges_to_test': 100,  # Test top N edges by flow
    'n_runs_per_edge': 10,
    'n_control_runs': 50,  # Baseline with intact network
    
    # Network parameters
    'n_cells': 50,
    'min_flow': 1.0,
    'barrier_height': 0.2,
    
    # Two-phase simulation parameters
    'cascade_duration': 200,
    'recovery_duration': 800,
    'dt': 0.5,
    'cascade_sigma': 0.06,
    'cascade_alpha': 1.5,
    'recovery_sigma': 0.04,
    'recovery_alpha': 2.0,
    
    # Seeds
    'base_seed': 42,
}

print("=" * 60)
print("EXPERIMENT 12: KEYSTONE EDGE ANALYSIS")
print("=" * 60)
print(f"Max edges to test: {SWEEP_CONFIG['max_edges_to_test']}")
print(f"Runs per edge: {SWEEP_CONFIG['n_runs_per_edge']}")
print(f"Control runs: {SWEEP_CONFIG['n_control_runs']}")

## 4. Network Creation and Edge Extraction

In [ ]:
def create_amazon_network_with_edge_info(data, config, seed=42):
    """
    Create Amazon network and return edge information for analysis.
    
    Returns:
        network: EnergyConstrainedNetwork
        edge_info: list of dicts with edge properties
    """
    np.random.seed(seed)
    
    network_matrix = data['network']
    n_cells = config['n_cells']
    min_flow = config['min_flow']
    barrier_height = config['barrier_height']
    
    total_flow = network_matrix.sum(axis=0) + network_matrix.sum(axis=1)
    top_indices = np.argsort(total_flow)[-n_cells:]
    
    net = EnergyConstrainedNetwork()
    
    # Add elements
    for i, idx in enumerate(top_indices):
        element = EnergyConstrainedCusp(
            a=-1.0, b=1.0, c=0.0, x_0=0.0,
            barrier_height=barrier_height,
            dissipation_rate=0.1
        )
        net.add_element(f'cell_{i}', element)
    
    # Add couplings and track edge info
    edge_info = []
    for i, idx_i in enumerate(top_indices):
        for j, idx_j in enumerate(top_indices):
            if i != j:
                flow = network_matrix[idx_i, idx_j]
                if flow > min_flow:
                    coupling = GradientDrivenCoupling(
                        conductivity=flow / 100.0,
                        state_coupling=0.1
                    )
                    net.add_coupling(f'cell_{i}', f'cell_{j}', coupling)
                    
                    edge_info.append({
                        'source': f'cell_{i}',
                        'target': f'cell_{j}',
                        'source_idx': i,
                        'target_idx': j,
                        'flow': flow,
                        'conductivity': flow / 100.0,
                    })
    
    return net, edge_info, top_indices


# Create baseline network
baseline_network, edge_info, selected_cells = create_amazon_network_with_edge_info(
    amazon_data, SWEEP_CONFIG
)

print(f"Baseline network: {baseline_network.n_elements} nodes, {len(edge_info)} edges")

# Sort edges by flow
edge_info_sorted = sorted(edge_info, key=lambda x: x['flow'], reverse=True)

# Select edges to test
n_edges_to_test = min(SWEEP_CONFIG['max_edges_to_test'], len(edge_info_sorted))
edges_to_test = edge_info_sorted[:n_edges_to_test]

print(f"\nEdges to test: {n_edges_to_test}")
print(f"Flow range: {edges_to_test[-1]['flow']:.2f} - {edges_to_test[0]['flow']:.2f} mm/month")

total_sims = SWEEP_CONFIG['n_control_runs'] + n_edges_to_test * SWEEP_CONFIG['n_runs_per_edge']
print(f"\nTotal simulations: {total_sims}")
print(f"Estimated runtime: ~{total_sims * 2 / 60:.0f} minutes on 14 workers")

In [ ]:
# Show top 10 edges by flow
print("\nTop 10 edges by moisture flow:")
print(f"{'Rank':<6} {'Source':<10} {'Target':<10} {'Flow (mm)':<12}")
print("-" * 40)
for i, edge in enumerate(edges_to_test[:10]):
    print(f"{i+1:<6} {edge['source']:<10} {edge['target']:<10} {edge['flow']:<12.2f}")

## 5. Worker Functions

In [ ]:
def run_edge_removal_experiment(args):
    """
    Worker function for single edge removal experiment.
    
    If edge_to_remove is None, runs with intact network (control).
    """
    network_bytes, edge_to_remove, config, seed = args
    
    import sys
    import numpy as np
    import pickle
    import copy
    
    if '/opt/research-local/src' not in sys.path:
        sys.path.insert(0, '/opt/research-local/src')
    
    from energy_constrained.solvers import run_two_phase_experiment
    
    # Reconstruct network
    baseline_network = pickle.loads(network_bytes)
    np.random.seed(seed)
    
    # Remove edge if specified
    if edge_to_remove is not None:
        # Create a copy and remove the edge
        network = copy.deepcopy(baseline_network)
        source = edge_to_remove['source']
        target = edge_to_remove['target']
        try:
            network.remove_coupling(source, target)
        except:
            pass  # Edge may not exist in this direction
    else:
        network = baseline_network
    
    # Run two-phase experiment
    result = run_two_phase_experiment(
        network=network,
        cascade_duration=config['cascade_duration'],
        recovery_duration=config['recovery_duration'],
        dt=config['dt'],
        cascade_sigma=config['cascade_sigma'],
        cascade_alpha=config['cascade_alpha'],
        recovery_sigma=config['recovery_sigma'],
        recovery_alpha=config['recovery_alpha'],
        seed=seed
    )
    
    # Compute metrics
    n_cells = result.x_full.shape[1]
    n_tip_events = 0
    n_recover_events = 0
    
    for j in range(n_cells):
        x_traj = result.x_full[:, j]
        signs = np.sign(x_traj)
        sign_changes = np.diff(signs)
        n_tip_events += np.sum(sign_changes > 0)
        n_recover_events += np.sum(sign_changes < 0)
    
    tip_recovery_ratio = n_tip_events / n_recover_events if n_recover_events > 0 else np.nan
    
    return {
        'edge_source': edge_to_remove['source'] if edge_to_remove else 'control',
        'edge_target': edge_to_remove['target'] if edge_to_remove else 'control',
        'edge_flow': edge_to_remove['flow'] if edge_to_remove else 0,
        'seed': seed,
        'recovery_fraction': result.metrics['recovery_fraction'],
        'n_tip_events': n_tip_events,
        'n_recover_events': n_recover_events,
        'tip_recovery_ratio': tip_recovery_ratio,
        'n_permanent_tips': result.metrics['n_permanent_tips'],
        'pct_tipped_cascade': result.metrics['pct_tipped_at_cascade_end'],
        'final_pct_tipped': result.metrics['final_pct_tipped'],
    }

print("Worker function defined.")

## 6. Run Edge Removal Sweep

In [ ]:
# Serialize and scatter baseline network
network_bytes = pickle.dumps(baseline_network)
print(f"Network serialized: {len(network_bytes) / 1024:.1f} KB")

network_future = client.scatter(network_bytes, broadcast=True)
print("Network broadcast to all workers")

# Build task arguments
print("\n" + "=" * 60)
print("EXPERIMENT 12: Starting Keystone Edge Sweep")
print("=" * 60)
start_time = time.time()

task_args = []

# Control runs (intact network)
for run_idx in range(SWEEP_CONFIG['n_control_runs']):
    seed = SWEEP_CONFIG['base_seed'] + run_idx
    task_args.append((network_bytes, None, SWEEP_CONFIG, seed))

# Edge removal runs
for edge_idx, edge in enumerate(edges_to_test):
    for run_idx in range(SWEEP_CONFIG['n_runs_per_edge']):
        seed = SWEEP_CONFIG['base_seed'] + 10000 + edge_idx * 100 + run_idx
        task_args.append((network_bytes, edge, SWEEP_CONFIG, seed))

print(f"Generated {len(task_args)} task arguments")
print(f"  Control runs: {SWEEP_CONFIG['n_control_runs']}")
print(f"  Edge removal runs: {n_edges_to_test * SWEEP_CONFIG['n_runs_per_edge']}")

# Submit all tasks
futures = client.map(run_edge_removal_experiment, task_args)
print(f"Submitted {len(futures)} tasks")

# Collect results
all_results = []
print("\nProgress:")
for i, future in enumerate(as_completed(futures)):
    result = future.result()
    all_results.append(result)
    
    if (i + 1) % 100 == 0:
        elapsed = time.time() - start_time
        rate = (i + 1) / elapsed
        remaining = (len(futures) - i - 1) / rate
        print(f"  Completed {i+1}/{len(futures)} ({100*(i+1)/len(futures):.1f}%) "
              f"- {elapsed:.0f}s elapsed, ~{remaining:.0f}s remaining")

elapsed = time.time() - start_time
print(f"\n" + "=" * 60)
print(f"COMPLETE: {len(all_results)} simulations in {elapsed:.1f}s ({elapsed/60:.1f} min)")
print("=" * 60)

## 7. Results Aggregation

In [ ]:
df = pd.DataFrame(all_results)
print(f"Results shape: {df.shape}")

# Create edge identifier
df['edge_id'] = df['edge_source'] + ' -> ' + df['edge_target']
df.loc[df['edge_source'] == 'control', 'edge_id'] = 'control'

df.head(10)

In [ ]:
# Summary statistics by edge
summary = df.groupby('edge_id').agg({
    'edge_flow': 'first',
    'recovery_fraction': ['mean', 'std'],
    'tip_recovery_ratio': ['mean', 'std'],
    'n_permanent_tips': ['mean', 'std'],
    'n_tip_events': 'mean',
    'n_recover_events': 'mean',
}).round(4)

summary.columns = ['_'.join(col) if col[1] else col[0] for col in summary.columns]
summary = summary.reset_index()

# Get control baseline
control = summary[summary['edge_id'] == 'control'].iloc[0]
control_recovery = control['recovery_fraction_mean']
control_ratio = control['tip_recovery_ratio_mean']

print(f"Control (intact network):")
print(f"  Recovery fraction: {control_recovery:.3f}")
print(f"  Tip/recovery ratio: {control_ratio:.3f}")

In [ ]:
# Calculate impact of each edge removal
summary['recovery_impact'] = summary['recovery_fraction_mean'] - control_recovery
summary['ratio_impact'] = summary['tip_recovery_ratio_mean'] - control_ratio

# Rank edges by negative impact (most harmful removals first)
edge_summary = summary[summary['edge_id'] != 'control'].copy()
edge_summary = edge_summary.sort_values('recovery_impact', ascending=True)
edge_summary['rank'] = range(1, len(edge_summary) + 1)

print("\nTop 20 KEYSTONE EDGES (most critical for recovery):")
print("=" * 80)
print(f"{'Rank':<6} {'Edge':<25} {'Flow':<10} {'Rec Impact':<12} {'Ratio Impact':<12}")
print("-" * 80)
for _, row in edge_summary.head(20).iterrows():
    print(f"{row['rank']:<6} {row['edge_id']:<25} {row['edge_flow_first']:<10.1f} "
          f"{row['recovery_impact']:<+12.3f} {row['ratio_impact']:<+12.3f}")

## 8. Visualization

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Panel 1: Recovery Impact Distribution
ax = axes[0, 0]
impacts = edge_summary['recovery_impact'].values
ax.hist(impacts, bins=30, color='steelblue', edgecolor='black', alpha=0.7)
ax.axvline(0, color='red', linestyle='--', linewidth=2, label='Control (no change)')
ax.axvline(np.percentile(impacts, 10), color='orange', linestyle=':', 
           label=f'10th percentile ({np.percentile(impacts, 10):.3f})')
ax.set_xlabel('Recovery Impact (vs Control)', fontsize=12)
ax.set_ylabel('Number of Edges', fontsize=12)
ax.set_title('Distribution of Edge Removal Impacts', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)

# Panel 2: Impact vs Flow
ax = axes[0, 1]
ax.scatter(edge_summary['edge_flow_first'], edge_summary['recovery_impact'],
           c=edge_summary['rank'], cmap='RdYlGn', s=50, alpha=0.7)
ax.axhline(0, color='gray', linestyle='--', alpha=0.7)
ax.set_xlabel('Edge Flow (mm/month)', fontsize=12)
ax.set_ylabel('Recovery Impact', fontsize=12)
ax.set_title('Impact vs Moisture Flow', fontsize=14)

# Highlight top 10 keystone edges
top10 = edge_summary.head(10)
ax.scatter(top10['edge_flow_first'], top10['recovery_impact'],
           s=150, facecolors='none', edgecolors='red', linewidths=2,
           label='Top 10 Keystone')
ax.legend()
ax.grid(True, alpha=0.3)

# Panel 3: Ranked Edge Impacts (bar chart)
ax = axes[1, 0]
top20 = edge_summary.head(20)
y_pos = range(len(top20))
colors = plt.cm.RdYlGn(np.linspace(0.1, 0.9, len(top20)))
ax.barh(y_pos, top20['recovery_impact'], color=colors, edgecolor='black')
ax.set_yticks(y_pos)
ax.set_yticklabels([f"{row['edge_id']} ({row['edge_flow_first']:.0f})" 
                    for _, row in top20.iterrows()], fontsize=8)
ax.axvline(0, color='red', linestyle='--')
ax.set_xlabel('Recovery Impact', fontsize=12)
ax.set_title('Top 20 Keystone Edges', fontsize=14)
ax.invert_yaxis()
ax.grid(True, alpha=0.3, axis='x')

# Panel 4: Asymmetry Impact (ratio change)
ax = axes[1, 1]
ax.scatter(edge_summary['recovery_impact'], edge_summary['ratio_impact'],
           c=edge_summary['edge_flow_first'], cmap='viridis', s=50, alpha=0.7)
ax.axhline(0, color='gray', linestyle='--', alpha=0.7)
ax.axvline(0, color='gray', linestyle='--', alpha=0.7)
ax.set_xlabel('Recovery Impact', fontsize=12)
ax.set_ylabel('Tip/Recovery Ratio Impact', fontsize=12)
ax.set_title('Recovery vs Asymmetry Impact', fontsize=14)
plt.colorbar(ax.collections[0], ax=ax, label='Edge Flow')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('/workspace/data/exp12_keystone_edges.png', dpi=150, bbox_inches='tight')
plt.show()
print("\nPlot saved to /workspace/data/exp12_keystone_edges.png")

## 9. Pattern Analysis

In [ ]:
# Analyze patterns in keystone edges
print("=" * 70)
print("KEYSTONE EDGE PATTERN ANALYSIS")
print("=" * 70)

# 1. Flow correlation
from scipy.stats import pearsonr, spearmanr

flow_corr, flow_p = pearsonr(edge_summary['edge_flow_first'], edge_summary['recovery_impact'])
flow_spearman, flow_sp = spearmanr(edge_summary['edge_flow_first'], edge_summary['recovery_impact'])

print(f"\n1. FLOW vs IMPACT CORRELATION:")
print(f"   Pearson r = {flow_corr:.3f} (p = {flow_p:.4f})")
print(f"   Spearman ρ = {flow_spearman:.3f} (p = {flow_sp:.4f})")

# 2. Node involvement analysis
print(f"\n2. NODE INVOLVEMENT IN TOP 20 KEYSTONE EDGES:")
top20 = edge_summary.head(20)

# Count node appearances
node_counts = defaultdict(int)
for _, row in top20.iterrows():
    source = row['edge_id'].split(' -> ')[0]
    target = row['edge_id'].split(' -> ')[1]
    node_counts[source] += 1
    node_counts[target] += 1

sorted_nodes = sorted(node_counts.items(), key=lambda x: x[1], reverse=True)
print("   Most common nodes in keystone edges:")
for node, count in sorted_nodes[:10]:
    print(f"     {node}: {count} appearances")

# 3. Directional patterns
print(f"\n3. FLOW DIRECTION ANALYSIS:")
# Check if high-impact edges tend to be in one direction
keystone_flows = top20['edge_flow_first'].mean()
other_flows = edge_summary.iloc[20:]['edge_flow_first'].mean()
print(f"   Mean flow of top 20 keystone edges: {keystone_flows:.1f} mm/month")
print(f"   Mean flow of other edges: {other_flows:.1f} mm/month")

In [ ]:
# Create connectivity impact matrix
print("\n" + "=" * 70)
print("CONNECTIVITY IMPACT MATRIX")
print("=" * 70)

# Build matrix of impacts
n_cells = SWEEP_CONFIG['n_cells']
impact_matrix = np.zeros((n_cells, n_cells))

for _, row in edge_summary.iterrows():
    if row['edge_id'] != 'control':
        parts = row['edge_id'].split(' -> ')
        source_idx = int(parts[0].replace('cell_', ''))
        target_idx = int(parts[1].replace('cell_', ''))
        impact_matrix[source_idx, target_idx] = row['recovery_impact']

# Plot impact matrix
fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(impact_matrix, cmap='RdBu', vmin=-0.1, vmax=0.1)
ax.set_xlabel('Target Cell', fontsize=12)
ax.set_ylabel('Source Cell', fontsize=12)
ax.set_title('Edge Removal Impact Matrix\n(Red = harmful, Blue = beneficial)', fontsize=14)
plt.colorbar(im, ax=ax, label='Recovery Impact')
plt.tight_layout()
plt.savefig('/workspace/data/exp12_impact_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

## 10. Key Findings Summary

In [ ]:
print("\n" + "=" * 70)
print("EXPERIMENT 12: KEY FINDINGS")
print("=" * 70)

# Statistics
n_harmful = (edge_summary['recovery_impact'] < -0.01).sum()
n_beneficial = (edge_summary['recovery_impact'] > 0.01).sum()
n_neutral = len(edge_summary) - n_harmful - n_beneficial

worst_edge = edge_summary.iloc[0]
best_edge = edge_summary.iloc[-1]

print(f"""
1. OVERALL STATISTICS:
   Edges tested: {len(edge_summary)}
   Harmful removals (< -1%): {n_harmful} ({100*n_harmful/len(edge_summary):.1f}%)
   Beneficial removals (> +1%): {n_beneficial} ({100*n_beneficial/len(edge_summary):.1f}%)
   Neutral: {n_neutral} ({100*n_neutral/len(edge_summary):.1f}%)

2. MOST CRITICAL EDGE (removing hurts recovery most):
   Edge: {worst_edge['edge_id']}
   Flow: {worst_edge['edge_flow_first']:.1f} mm/month
   Recovery impact: {worst_edge['recovery_impact']:+.3f}

3. LEAST CRITICAL EDGE (removing may help recovery):
   Edge: {best_edge['edge_id']}
   Flow: {best_edge['edge_flow_first']:.1f} mm/month
   Recovery impact: {best_edge['recovery_impact']:+.3f}

4. FLOW-IMPACT CORRELATION:
   r = {flow_corr:.3f} (p = {flow_p:.4f})
   {"Strong" if abs(flow_corr) > 0.5 else "Moderate" if abs(flow_corr) > 0.3 else "Weak"} correlation between flow and criticality

5. CONSERVATION IMPLICATIONS:
   Protecting the top 10 keystone edges would preserve:
   - Maximum recovery potential
   - Nodes involved: {', '.join([n for n, c in sorted_nodes[:5]])}
""")

## 11. Save Results

In [ ]:
df.to_csv('/workspace/data/experiment12_keystone_edges_full.csv', index=False)
print(f"Full results saved to /workspace/data/experiment12_keystone_edges_full.csv")

edge_summary.to_csv('/workspace/data/experiment12_keystone_edges_summary.csv', index=False)
print(f"Summary saved to /workspace/data/experiment12_keystone_edges_summary.csv")

# Save keystone ranking
keystone_ranking = edge_summary[['rank', 'edge_id', 'edge_flow_first', 
                                  'recovery_impact', 'ratio_impact']].head(50)
keystone_ranking.to_csv('/workspace/data/experiment12_keystone_ranking.csv', index=False)
print(f"Keystone ranking saved to /workspace/data/experiment12_keystone_ranking.csv")

In [ ]:
print("\n" + "=" * 70)
print("EXPERIMENT 12 COMPLETE")
print("=" * 70)

print(f"""
CONFIGURATION:
- Edges tested: {len(edge_summary)}
- Runs per edge: {SWEEP_CONFIG['n_runs_per_edge']}
- Control runs: {SWEEP_CONFIG['n_control_runs']}
- Total simulations: {len(df)}
- Runtime: {elapsed:.1f}s ({elapsed/60:.1f} min)

FILES GENERATED:
- /workspace/data/experiment12_keystone_edges_full.csv
- /workspace/data/experiment12_keystone_edges_summary.csv
- /workspace/data/experiment12_keystone_ranking.csv
- /workspace/data/exp12_keystone_edges.png
- /workspace/data/exp12_impact_matrix.png

NEXT STEPS:
- Use keystone ranking to prioritize conservation efforts
- Test combined removal of multiple keystone edges
- Map keystone edges to geographic locations
""")